# Tutorial: Combinators (One-Shot `evaluate(...)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/combinator.html

This is the one-shot variant: each line is an independent `evaluate("...")` call.


In [36]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import evaluate

print(f"Using bqn_jax from: {bqn_jax.__file__}")


Using bqn_jax from: /home/johtok/repos/jax/xlapl/bqn-jax/src/bqn_jax/__init__.py


## Before You Start

1. Run the setup cell first (`from bqn_jax import evaluate`).
2. Then run cells from top to bottom.
3. Each `evaluate("...")` call parses and runs only that snippet.
4. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## What's a combinator?

Combinators build new functions from existing ones without introducing explicit variables.


In [37]:
evaluate('|∘- 6')


Array(6., dtype=float32, weak_type=True)

In [38]:
evaluate('7 |∘- 9')


Array(2., dtype=float32, weak_type=True)

In [39]:
evaluate('14‿8 |∘- 19‿6')
evaluate('14‿8 +´∘|∘- 19‿6')


Array(7., dtype=float32)

## Comparisons

Use these examples to see scalar and elementwise boolean comparisons.


In [40]:
evaluate('3 < 4')
evaluate('4 > ∞')
evaluate('∞ < @')


Array(0, dtype=int32)

In [41]:
evaluate('\'e\' = "George Boole"')
evaluate('+´ \'e\' = "George Boole"')
evaluate('\'e\' +´∘= "George Boole"  # With a combinator')


Array(3, dtype=int32)

In [42]:
evaluate('"abcd" ×´∘= "abdd"')


Array(0, dtype=int32)

In [43]:
evaluate('"abcd" ≡ "abdd"')
evaluate('"abc"‿"de" ≡ "abc"‿"de"')


Array(1, dtype=int32)

In [44]:
evaluate('2‿3‿4‿2 ≠ 3‿3‿2‿2')
evaluate('2‿3‿4‿2 ≢ 3‿3‿2‿2')


Array([4], dtype=int32)

## Length, rank, and depth

These primitives inspect structure: item count, dimensionality, and nesting depth.


In [45]:
evaluate('≠ "testing"')
evaluate('≠ ⟨⟩')
evaluate('≠ ⟨ π, ∘, "element" ⋄ ⟨\'l\',1,5,\'t\'⟩ ⟩')
evaluate('≠ 4')


Array(1, dtype=int32)

In [46]:
evaluate('= 0.5')
evaluate('= ↕3')
evaluate("= 'a'")
evaluate('= "a"')


Array(1, dtype=int32)

In [47]:
evaluate('≡ "dream"                  # An ordinary dream')
evaluate('≡ "d"‿"r"‿"e"‿"a"‿"m"      # What if the letters were strings?')
evaluate('≡ ⟨ "d"‿"r"‿"e"‿"a"‿"m" ⟩  # We have to go deeper')


Array(3, dtype=int32)

## Composition

Composition combines smaller transforms into larger reusable pipelines.


In [48]:
evaluate('(≠"string") = ≠"sting"')


Array(0, dtype=int32)

In [49]:
evaluate('=´≠¨ ⟨"string","sting"⟩')


Array(0, dtype=int32)

In [50]:
evaluate('"string" =○≠ "sting"')


Array(0, dtype=int32)

In [51]:
evaluate('"string" ⋈○≠ "sting"')
evaluate('⋈○≠ "sting"')


Array([5], dtype=int32)

In [52]:
evaluate('2 ⋆⟜- 3')
evaluate('2 ⋆⊸- 3')


Array(4.389056, dtype=float32, weak_type=True)

In [53]:
evaluate('4 -⊸⌽ " before"  # Rotate to the right by four')
evaluate('4 ⌽⁼  " before"  # Okay this time Undo is better')


Array([102, 111, 114, 101,  32,  98, 101], dtype=int32)

In [54]:
evaluate('¬⊸× 0.5')


Array(0.25, dtype=float32, weak_type=True)

In [55]:
evaluate('↕⊸÷ 8')
evaluate('¬⊸× ↕⊸÷ 8')


Array([0.      , 0.109375, 0.1875  , 0.234375, 0.25    , 0.234375,
       0.1875  , 0.109375], dtype=float32, weak_type=True)

In [56]:
evaluate('1⊸+ 5')
evaluate('+⟜1 5')


Array(6., dtype=float32, weak_type=True)

In [57]:
evaluate('"const"˜ 5')
evaluate('@ "const"˜ 6')


Array([ 99, 111, 110, 115, 116], dtype=int32)

In [58]:
evaluate('+⊸1 5')


Array(1., dtype=float32, weak_type=True)

In [59]:
evaluate('↕⊸÷ 8')
evaluate('(↕8) ÷ 7')


Array([0.        , 0.14285715, 0.2857143 , 0.42857143, 0.5714286 ,
       0.71428573, 0.85714287, 1.        ], dtype=float32, weak_type=True)

In [60]:
evaluate('-⟜1 8')


Array(7., dtype=float32, weak_type=True)

In [61]:
evaluate('↕⊸÷⟜(-⟜1) 8')


Array([0.        , 0.14285715, 0.2857143 , 0.42857143, 0.5714286 ,
       0.71428573, 0.85714287, 1.        ], dtype=float32, weak_type=True)

In [62]:
evaluate('(↕÷-⟜1) 8')


Array([0.        , 0.14285715, 0.2857143 , 0.42857143, 0.5714286 ,
       0.71428573, 0.85714287, 1.        ], dtype=float32, weak_type=True)

## Base decoding continued

This revisits decoding and progressively compresses it into tacit combinator form.


In [63]:
evaluate('@ + +´¨ (⌽2⋆↕8)⊸×¨ \'0\' -˜ "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [64]:
evaluate('@⊸+ +´¨ (⌽2⋆↕8)⊸×¨ -⟜\'0\' "01000010"‿"01010001"‿"01001110"')
evaluate('(@⊸+)∘(+´¨)∘((⌽2⋆↕8)⊸×¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [65]:
evaluate('@⊸+∘(+´∘((⌽2⋆↕8)⊸×)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [66]:
evaluate('@⊸+∘(+´∘×⟜(⌽2⋆↕8)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [67]:
evaluate('(((((1×10)+2)×10)+3)×10)+4')
evaluate('((1×⟜10⊸+2)×⟜10⊸+3)×⟜10⊸+4   # Make the combining step a function')
evaluate('4+⟜(10⊸×)3+⟜(10⊸×)2+⟜(10⊸×)1 # Flip the combining function around')
evaluate("+⟜(10⊸×)´ 4‿3‿2‿1            # Now it's a BQN fold")
evaluate('+⟜(10⊸×)´ ⌽ 1‿2‿3‿4          # To fold in reverse, reverse then fold')


Array(1234., dtype=float32, weak_type=True)

In [68]:
evaluate('+´∘×⟜(⌽2⋆↕8) "01010001"-\'0\'')
evaluate('+⟜(2⊸×)´∘⌽ "01010001"-\'0\'')


Array(81., dtype=float32)

In [69]:
evaluate('@⊸+∘(+⟜(2⊸×)´∘⌽¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [70]:
evaluate('(@+ ·+⟜(2⊸×)´∘⌽¨ -⟜\'0\') "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

## Summary

If any result is unclear, rerun the section above step by step before moving on.
